In [ ]:
import os
os.environ['JDBC_HOST'] = 'jrtest01-splice-hregion'

<link rel="stylesheet" href="https://doc.splicemachine.com/zeppelin/css/zepstyles2.css" />

# Transactions with Spark and JDBC
This notebook introduces using Spark and Scala to programmatically interact with Splice Machine via a JDBC connection. In this notebook, we'll:

1. *Run setup code to create our variables and define a simple Spark procedure named `checkBalance`.*
2. *Use JDBC calls to set and check values from the database.*
3. *Show the use of `commit` and `rollback` to control transactional persistence of our updates.*
<br />


## Run Setup Code and Define our Procedure

First, we configure the JDBC URL we'll use to connect to your Splice Machine database:

In [ ]:
%%scala 

val localJDBCURL = """jdbc:splice://localhost:1527/splicedb;user=splice;password=admin"""


<br />
Next, we define a simple procedure, `checkBalance` that checks and *pretty-prints* the current balance for a supplier:

In [ ]:
%%scala 
import java.sql.{DriverManager,Connection}
var conn = DriverManager.getConnection(localJDBCURL)
conn.setAutoCommit(false)
var statement = conn.createStatement()

def checkBalance(suppKey: Integer) : Unit = {
    var resSet = statement.executeQuery("SELECT S_ACCTBAL FROM DEV2.SUPPLIER WHERE S_SUPPKEY = " + suppKey.toString())
    while ( resSet.next() ) {
        var bl = resSet.getString("S_ACCTBAL")
        println("================ Current account balance: " + bl + " ====================");
    }
}


Now we can check the balance easily at any time with the checkBalance call, passing in the supply key as a parameter:

In [ ]:
%%scala 
checkBalance(6517)

Our setup code created two useful variables we can now employ:

<table class="splicezepNoBorder">
   <tbody>
      <tr>
          <td><code>statement</code></td>
          <td>The standard JDBC Statement object, on which we run functions such as <code>executeUpdate</code>.
      </tr>
      <tr style="background-color: transparent;">
          <td><code>conn</code></td>
          <td style="background-color: transparent;" >A standard JDBC Connection object, on which run <code>commit</code> and <code>rollback</code>.
      </tr>
    </tbody>
</table>


## Committing a Transaction

The first example shows an update statement which, upon committing, retains the value to which it is set.


In [ ]:
%%scala 
statement.executeUpdate("update DEV2.SUPPLIER SET S_ACCTBAL = 10000 WHERE S_SUPPKEY = 6517");
checkBalance(6517);
conn.commit();
checkBalance(6517);

## Rolling Back a Transaction

If we decide to do a rollback, the value returns to the previous value before the transactional context, as shown in the next example:



In [ ]:
%%scala 
statement.executeUpdate("update DEV2.SUPPLIER SET S_ACCTBAL = 0 WHERE S_SUPPKEY = 6517")
checkBalance(6517);
conn.rollback();
checkBalance(6517);


 
<br />
This was a simple demonstration of transactional commit and rollback, even from within Scala code.  Later on we'll get more involved with Spark libraries using the Native Spark Datasource.


## Where to Go Next

The next notebook in this class introduces you to [*Using our Native Spark DataSource*](./f.%20Using%20our%20Native%20Spark%20DataSource.ipynb) to work with your database.